In [ ]:
import ee
from opticalflood import opticflood

: 

In [ ]:
def initialize_earth_engine(gee_project_name):
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=gee_project_name, opt_url='https://earthengine-highvolume.googleapis.com')
initialize_earth_engine('meta-shape-468218-g1')

In [ ]:
# --- AOI and Parameters ---
aoi = ee.Geometry.Polygon([[
    [11.705864, 44.607065],
    [11.95545, 44.58197],
    [12.24048, 44.55463],
    [12.23305, 44.50447],
    [11.96129, 44.52889],
    [11.69375, 44.55232],
    [11.705864, 44.607065]  # close the polygon
]])
flood_event_date = "2023-05-18"
lag = 20
cloud_threshold=50,
min_area = 1000

In [ ]:
flood_maps_s2 = opticflood(aoi,flood_event_date,lag , min_area=min_area)

In [ ]:
import geemap
import ee

Map = geemap.Map()

# --- Visualization palette (0 = dry, 1 = wet/flood) ---
flood_vis = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#0000ff']  # white (dry) to blue (flooded)

}

# --- Convert ImageCollection to list ---
flood_list = flood_maps_s2.toList(flood_maps_s2.size())
count = flood_maps_s2.size().getInfo()

# --- Add each flood mask to the map ---
for i in range(count):
    img = ee.Image(flood_list.get(i))
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    # Clip to AOI for display
    mask_clipped = img.select('depth').clip(aoi)  # <--- Only this line is changed

    # Add flood classification
    Map.addLayer(mask_clipped, flood_vis, f'Flood {date}')

# Center and display
Map.centerObject(aoi, 10)
Map


In [ ]:
import ee
# ee.Initialize()

# === Settings ===
drive_folder = "GEE_Flood_Exports_tot"   # Folder that will appear in your Google Drive
scale_m = 10                         # 10 m resolution

# Convert collection to a list and get size
flood_list = flood_maps_s2.toList(flood_maps_s2.size())
n = flood_maps_s2.size().getInfo()

tasks = []
for i in range(n):
    img = ee.Image(flood_list.get(i))

    # Use image date to make a unique filename (e.g., 20250810)
    date_str = ee.Date(img.get('system:time_start')).format('YYYYMMdd').getInfo()

    # Prepare export image: select flood band, clip to AOI, make a compact byte raster
    out = (img.select('depth')      # <- your flood band
              .clip(aoi)
              .unmask(0)            # fill NoData with 0
              .rename('flood')
              .toByte())

    task = ee.batch.Export.image.toDrive(
        image=out,
        description=f"flood_{date_str}",
        folder=drive_folder,
        fileNamePrefix=f"flood_{date_str}",
        region=aoi,                 # you can also pass aoi.bounds() if you prefer
        scale=scale_m,              # 10 m
        maxPixels=1e13
        # Optional: set a CRS explicitly if you need to force one
        # crs='EPSG:32632',
    )
    task.start()
    tasks.append(task)

print(f"Started {len(tasks)} export task(s).")
for t in tasks:
    print("TaskID:", t.id, "| Description:", t.config.get('description'))

https://code.earthengine.google.com/tasks
#the link of google earth engine takes to check the process